In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns


In [7]:
df=pd.read_csv("/home/sachchida/gitrepo/ml/data/hate-speech/labeled_data.csv")
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [8]:
import re
def clean_text(text):
    # remove HTML
    text = re.sub(r'<.*?>', '', text)
    # replace URLs
    text = re.sub(r'http\S+|www\S+', ' URL ', text)
    # replace user mentions (handles @name and @name:)
    text = re.sub(r'@\w+:?', ' USER ', text)
    # keep hashtag text
    text = re.sub(r'#(\w+)', r'\1', text)
    # lowercase
    text = text.lower()
    # remove non-alphabetic characters
    text = re.sub(r'[^a-z\s]', '', text)
    # normalize spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text
import re
def clean_text_with_caps(text):
    text = re.sub(r'<.*?>', '', text)   # HTML
    text = re.sub(r'http\S+|www\S+', ' <URL> ', text)
    text = re.sub(r'@\w+', ' <USER> ', text)
    text = re.sub(r'#(\w+)', r'\1', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # keep caps
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [9]:
df['tweet_cleaned']=df['tweet'].apply(clean_text)
df['tweet_cleaned_caps']=df['tweet'].apply(clean_text_with_caps)

In [136]:
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet,tweet_cleaned,tweet_cleaned_caps
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,rt user as a woman you shouldnt complain about...,RT USER As a woman you shouldnt complain about...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,rt user boy dats coldtyga dwn bad for cuffin d...,RT USER boy dats coldtyga dwn bad for cuffin d...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,rt user dawg rt user you ever fuck a bitch and...,RT USER Dawg RT USER You ever fuck a bitch and...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,rt user user she look like a tranny,RT USER USER she look like a tranny
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,rt user the shit you hear about me might be tr...,RT USER The shit you hear about me might be tr...


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

word_tfidf=TfidfVectorizer(
    analyzer="word",
    ngram_range=(1,2),
    stop_words='english',
    min_df=3, ## ignore terms (words) that appear in fewer than 3 different documents
    max_df=0.9 ## ignore terms that appear in more than 90% of the documents
)
char_tfidf=TfidfVectorizer(
    analyzer='char',
    ngram_range=(3,5),
)

In [138]:
df=df.drop(['Unnamed: 0','count','hate_speech','offensive_language','neither'],axis=1)
df.head()

,class,tweet,tweet_cleaned,tweet_cleaned_caps
0,2,!!! RT @mayasolovely: As a woman you shouldn't...,rt user as a woman you shouldnt complain about...,RT USER As a woman you shouldnt complain about...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,rt user boy dats coldtyga dwn bad for cuffin d...,RT USER boy dats coldtyga dwn bad for cuffin d...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,rt user dawg rt user you ever fuck a bitch and...,RT USER Dawg RT USER You ever fuck a bitch and...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,rt user user she look like a tranny,RT USER USER she look like a tranny
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,rt user the shit you hear about me might be tr...,RT USER The shit you hear about me might be tr...


In [11]:
from sklearn.model_selection import train_test_split
X=df['tweet_cleaned']
y=df['class']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)


In [12]:
X_train_tfidf_word=word_tfidf.fit_transform(X_train)
X_test_tfidf_word=word_tfidf.transform(X_test)

In [13]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(
    max_iter=1000,
    class_weight='balanced',
    n_jobs=-1
)
model.fit(X_train_tfidf_word,y_train)

/home/sachchida/anaconda3/envs/agentic-ai/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",'balanced'
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :ter

In [142]:
from sklearn.metrics import classification_report,confusion_matrix

y_pred=model.predict(X_test_tfidf_word)

In [143]:
from sklearn.metrics import classification_report,f1_score,confusion_matrix

In [144]:
print(f"classification_report",classification_report(y_test,y_pred))
print(f"Confusion Matrix ",confusion_matrix(y_test,y_pred))
print(f"Macro Average F1 Score ",f1_score(y_test,y_pred,average="macro"))
print(f"Weighted Average F1 Score ",f1_score(y_test,y_pred,average="weighted"))

classification_report               precision    recall  f1-score   support

           0       0.32      0.60      0.42       290
           1       0.97      0.86      0.91      3832
           2       0.76      0.92      0.83       835

    accuracy                           0.86      4957
   macro avg       0.68      0.79      0.72      4957
weighted avg       0.90      0.86      0.87      4957

Confusion Matrix  [[ 173   76   41]
 [ 330 3300  202]
 [  31   33  771]]
Macro Average F1 Score  0.7217811776923838
Weighted Average F1 Score  0.8696609400993828


In [145]:
from sklearn.svm import LinearSVC
svm=LinearSVC()
svm.fit(X_train_tfidf_word,y_train)
y_pred_svm=svm.predict(X_test_tfidf_word)

In [146]:
print(f"classification_report",classification_report(y_test,y_pred_svm))
print(f"Confusion Matrix ",confusion_matrix(y_test,y_pred_svm))
print(f"Macro Average F1 Score ",f1_score(y_test,y_pred_svm,average="macro"))
print(f"Weighted Average F1 Score ",f1_score(y_test,y_pred_svm,average="weighted"))

classification_report               precision    recall  f1-score   support

           0       0.45      0.25      0.32       290
           1       0.92      0.95      0.94      3832
           2       0.83      0.84      0.84       835

    accuracy                           0.89      4957
   macro avg       0.73      0.68      0.70      4957
weighted avg       0.88      0.89      0.88      4957

Confusion Matrix  [[  73  188   29]
 [  74 3642  116]
 [  16  114  705]]
Macro Average F1 Score  0.6986064843422541
Weighted Average F1 Score  0.8839485634757537


In [148]:
from sklearn.svm import SVC
svc_rbf=SVC(kernel='rbf', gamma='scale')
svc_rbf.fit(X_train_tfidf_word,y_train)
y_pred_svc_rbf=svc_rbf.predict(X_test_tfidf_word)

In [ ]:
print(f"classification_report",classification_report(y_test,y_pred_svm))
print(f"Confusion Matrix ",confusion_matrix(y_test,y_pred_svm))
print(f"Macro Average F1 Score ",f1_score(y_test,y_pred_svm,average="macro"))
print(f"Weighted Average F1 Score ",f1_score(y_test,y_pred_svm,average="weighted"))

classification_report               precision    recall  f1-score   support

           0       0.45      0.25      0.32       290
           1       0.92      0.95      0.94      3832
           2       0.83      0.84      0.84       835

    accuracy                           0.89      4957
   macro avg       0.73      0.68      0.70      4957
weighted avg       0.88      0.89      0.88      4957

Confusion Matrix  [[  73  188   29]
 [  74 3642  116]
 [  16  114  705]]
Macro Average F1 Score  0.6986064843422541
Weighted Average F1 Score  0.8839485634757537


### since F1 score for class 0 is low we shold try with Char_tfidf

In [ ]:
X_train_tfidf_char=char_tfidf.fit_transform(X_train)
X_test_tfidf_char=char_tfidf.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
model_lr_char=LogisticRegression(
    max_iter=1000,
    class_weight='balanced',
    solver='lbfgs',
    n_jobs=-1
)
model_lr_char.fit(X_train_tfidf_char,y_train)

/home/sachchida/anaconda3/envs/agentic-ai/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",'balanced'
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :ter

In [ ]:
y_pred_lr_char=model_lr_char.predict(X_test_tfidf_char)

In [ ]:
print(f"classification_report",classification_report(y_test,y_pred_lr_char))
print(f"Confusion Matrix ",confusion_matrix(y_test,y_pred_lr_char))
print(f"Macro Average F1 Score ",f1_score(y_test,y_pred_lr_char,average="macro"))
print(f"Weighted Average F1 Score ",f1_score(y_test,y_pred_lr_char,average="weighted"))

classification_report               precision    recall  f1-score   support

           0       0.34      0.56      0.42       290
           1       0.97      0.87      0.92      3832
           2       0.75      0.93      0.83       835

    accuracy                           0.86      4957
   macro avg       0.69      0.79      0.72      4957
weighted avg       0.89      0.86      0.87      4957

Confusion Matrix  [[ 161   77   52]
 [ 290 3333  209]
 [  18   37  780]]
Macro Average F1 Score  0.7238613542538347
Weighted Average F1 Score  0.8728401496369034


## somewhat improved try SVC

In [ ]:
svm_char=LinearSVC()
svm_char.fit(X_train_tfidf_char,y_train)
y_pred_svm_char=svm_char.predict(X_test_tfidf_char)

In [ ]:
print(f"classification_report",classification_report(y_test,y_pred_svm_char))
print(f"Confusion Matrix ",confusion_matrix(y_test,y_pred_svm_char))
print(f"Macro Average F1 Score ",f1_score(y_test,y_pred_svm_char,average="macro"))
print(f"Weighted Average F1 Score ",f1_score(y_test,y_pred_svm_char,average="weighted"))

classification_report               precision    recall  f1-score   support

           0       0.52      0.22      0.31       290
           1       0.92      0.96      0.94      3832
           2       0.83      0.84      0.83       835

    accuracy                           0.89      4957
   macro avg       0.75      0.67      0.69      4957
weighted avg       0.88      0.89      0.88      4957

Confusion Matrix  [[  65  192   33]
 [  53 3665  114]
 [   8  129  698]]
Macro Average F1 Score  0.6936774415573342
Weighted Average F1 Score  0.8830495425517101


### LR performing better than SVC will use mix of char+word tfidf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

word_tfidf = TfidfVectorizer(
    analyzer='word',
    ngram_range=(1, 2),
    min_df=3,
    max_df=0.9,
    stop_words='english'
)

char_tfidf = TfidfVectorizer(
    analyzer='char',
    ngram_range=(3, 5),
    min_df=3
)

In [ ]:
from sklearn.pipeline import FeatureUnion

tfidf_union = FeatureUnion([
    ('word_tfidf', word_tfidf),
    ('char_tfidf', char_tfidf)
])

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

pipeline = Pipeline([
    ('tfidf', tfidf_union),
    ('clf', LogisticRegression(
        max_iter=1000,
        class_weight='balanced',
        n_jobs=-1
    ))
])


In [ ]:
pipeline.fit(X_train, y_train)
y_pred_pipeline = pipeline.predict(X_test)

/home/sachchida/anaconda3/envs/agentic-ai/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
print(f"classification_report",classification_report(y_test,y_pred_pipeline))
print(f"Confusion Matrix ",confusion_matrix(y_test,y_pred_pipeline))
print(f"Macro Average F1 Score ",f1_score(y_test,y_pred_pipeline,average="macro"))
print(f"Weighted Average F1 Score ",f1_score(y_test,y_pred_pipeline,average="weighted"))

classification_report               precision    recall  f1-score   support

           0       0.36      0.54      0.43       290
           1       0.97      0.89      0.93      3832
           2       0.78      0.93      0.85       835

    accuracy                           0.87      4957
   macro avg       0.70      0.78      0.73      4957
weighted avg       0.90      0.87      0.88      4957

Confusion Matrix  [[ 156   89   45]
 [ 249 3406  177]
 [  26   34  775]]
Macro Average F1 Score  0.7347399757925647
Weighted Average F1 Score  0.8832280613786039


### Using BERT

In [39]:
import re

def clean_for_bert(text):
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    return text.strip()

### TFIDF can be replaced by Tokenization

In [40]:
from transformers import BertTokenizer
tokenizer=BertTokenizer.from_pretrained("bert-base-uncased")
def tokenize(batch):
    return tokenizer(
        batch['tweet'],
        padding='max_length',
        truncation=True,
        max_length=128
    )

In [41]:
from datasets import Dataset, ClassLabel

dataset = Dataset.from_pandas(
    df[['tweet', 'class']].rename(columns={'class': 'labels'})
)

dataset = dataset.cast_column(
    'labels',
    ClassLabel(num_classes=3, names=['hate', 'offensive', 'neither'])
)

dataset = dataset.train_test_split(
    test_size=0.2,
    stratify_by_column='labels',
    seed=42
)


Casting the dataset:   0%|          | 0/24783 [00:00<?, ? examples/s]

In [42]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=3
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Why are we using class_weights?
| Class | Meaning   | Frequency |
| ----- | --------- | --------- |
| 0     | hate      | ~6% ❌     |
| 1     | offensive | ~75%      |
| 2     | neither   | ~19%      |
Without class weights:

Model learns: “predict offensive most of the time”

Minority class (hate) is ignored

High accuracy, poor macro F1
By default, CrossEntropyLoss is: Loss = −log(P(true_class))
With class weights: Loss = − weight[class] × log(P(true_class)) so we use class_weights = [3.0, 1.0, 1.5]
we manually set loss_fn on the model
loss_fn = CrossEntropyLoss(weight=class_weights)
model.loss_fn = loss_fn

This is same as class weight in logistic regression. LogisticRegression(class_weight='balanced')



In [ ]:
import torch
from torch.nn import CrossEntropyLoss

class_weights = torch.tensor([3.0, 1.0, 1.5]).to('cuda')

loss_fn = CrossEntropyLoss(weight=class_weights)
model.loss_fn = loss_fn


In [34]:
from transformers import Trainer, TrainingArguments
import accelerate
print(accelerate.__version__)

1.12.0


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',
    save_strategy='epoch',         

    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,

    learning_rate=2e-5,
    weight_decay=0.01,

    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model='macro_f1',
    greater_is_better=True
)


In [45]:
from sklearn.metrics import f1_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=1)
    return {
        'macro_f1': f1_score(labels, preds, average='macro')
    }


In [46]:
tokenized_dataset = dataset.map(tokenize, batched=True)

tokenized_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"]
)


Map:   0%|          | 0/19826 [00:00<?, ? examples/s]

Map:   0%|          | 0/4957 [00:00<?, ? examples/s]

In [47]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    compute_metrics=compute_metrics
)

trainer.train()


Epoch,Training Loss,Validation Loss,Macro F1
1,0.267700,0.255324,0.731658
2,0.219600,0.266144,0.687399
3,0.152300,0.283101,0.769434


TrainOutput(global_step=3720, training_loss=0.22862443629131524, metrics={'train_runtime': 140.3636, 'train_samples_per_second': 423.742, 'train_steps_per_second': 26.503, 'total_flos': 3912364964906496.0, 'train_loss': 0.22862443629131524, 'epoch': 3.0})

In [49]:
predictions = trainer.predict(tokenized_dataset["test"])


In [50]:
import numpy as np

y_true = predictions.label_ids
y_pred = np.argmax(predictions.predictions, axis=1)


In [52]:
from sklearn.metrics import classification_report

print(
    classification_report(
        y_true,
        y_pred,
        target_names=["hate", "offensive", "neither"]
    )
)
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred)
print(cm)
from sklearn.metrics import f1_score

macro_f1 = f1_score(y_true, y_pred, average="macro")
weighted_f1 = f1_score(y_true, y_pred, average="weighted")

print("Macro F1:", macro_f1)
print("Weighted F1:", weighted_f1)



              precision    recall  f1-score   support

        hate       0.52      0.41      0.46       286
   offensive       0.94      0.96      0.95      3838
     neither       0.90      0.89      0.90       833

    accuracy                           0.92      4957
   macro avg       0.79      0.75      0.77      4957
weighted avg       0.91      0.92      0.91      4957

[[ 118  155   13]
 [  92 3678   68]
 [  16   73  744]]
Macro F1: 0.7694336739059638
Weighted F1: 0.9128750261663721


### Lets improve this further by using class_weights [4.0, 1.0, 1.5]

In [53]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=3
)
class_weights = torch.tensor([4.0, 1.0, 1.5]).to('cuda')

loss_fn = CrossEntropyLoss(weight=class_weights)
model.loss_fn = loss_fn

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [54]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='/home/sachchida/gitrepo/ml/data/results',
    eval_strategy='epoch',
    save_strategy='epoch',         

    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5, ## increased epoch from 3->5

    learning_rate=2e-5,
    weight_decay=0.01,

    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model='macro_f1',
    greater_is_better=True
)


In [55]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    compute_metrics=compute_metrics
)

trainer.train()

Epoch,Training Loss,Validation Loss,Macro F1
1,0.273000,0.262494,0.756704
2,0.224900,0.284514,0.644584
3,0.153200,0.301401,0.757404
4,0.103700,0.381540,0.764830
5,0.083200,0.429977,0.759661


TrainOutput(global_step=6200, training_loss=0.17713995656659526, metrics={'train_runtime': 232.0811, 'train_samples_per_second': 427.135, 'train_steps_per_second': 26.715, 'total_flos': 6520608274844160.0, 'train_loss': 0.17713995656659526, 'epoch': 5.0})

In [56]:
predictions = trainer.predict(tokenized_dataset["test"])
import numpy as np

y_true = predictions.label_ids
y_pred = np.argmax(predictions.predictions, axis=1)

In [57]:
from sklearn.metrics import classification_report

print(
    classification_report(
        y_true,
        y_pred,
        target_names=["hate", "offensive", "neither"]
    )
)
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred)
print(cm)
from sklearn.metrics import f1_score

macro_f1 = f1_score(y_true, y_pred, average="macro")
weighted_f1 = f1_score(y_true, y_pred, average="weighted")

print("Macro F1:", macro_f1)
print("Weighted F1:", weighted_f1)



              precision    recall  f1-score   support

        hate       0.53      0.40      0.46       286
   offensive       0.94      0.96      0.95      3838
     neither       0.90      0.88      0.89       833

    accuracy                           0.91      4957
   macro avg       0.79      0.75      0.76      4957
weighted avg       0.91      0.91      0.91      4957

[[ 114  160   12]
 [  85 3686   67]
 [  15   87  731]]
Macro F1: 0.7648303077291674
Weighted F1: 0.9103465776618838
